# GOOGLENET 

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, concatenate
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from PIL import Image


In [2]:
# 데이터 전처리 (이미지 사이즈 변경)
# 하나의 이미지를 전처리 할 때 사용 
def preprocess_image(image_path, img_width, img_height):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))
    img = img.astype('float32') / 255.0 # 픽셀 값 [0,1] 사이로 정규화 
    return img

# def preprocess_image(image_path, img_width, img_height):
#     try:
#         img = Image.open(image_path)
#         img = img.resize((img_width, img_height))
#         img = np.array(img)
#         return img
#     except Exception as e:
#         print(f"Error processing {image_path}: {e}")
#         return None



In [3]:
# datasets 안에 있는 감정이름으로 된 subdirectory 안에 있는 이미지 불러오기 
def load_dataset(data_dir, img_width, img_height):
    X = []
    y = []

    emotion_classes = sorted(os.listdir(data_dir))
    for i, emotion in enumerate(emotion_classes):
        emotion_dir = os.path.join(data_dir, emotion)

        if os.path.isdir(emotion_dir):
            for image_name in os.listdir(emotion_dir):
                image_path = os.path.join(emotion_dir, image_name)
                try:
                    img = preprocess_image(image_path, img_width, img_height) # 이미지 전처리 
                    X.append(img) # 전처리된 이미지 어레이에 넣기
                    y.append(i) # 해당 이미지의 감정 라벨(ex. happy, neutral) 어레이에 넣기 
                except Exception as e:
                    print(f"Error: {e}, Skipping image: {image_path}") 

    X = np.array(X)
    y = np.array(y)
    y = to_categorical(y, num_classes=len(emotion_classes)) # categorical_crossentropy (원핫 인코딩으로 분류를 하기 위함)
    return X, y


In [4]:

from tensorflow.keras.layers import Concatenate

def inception_module(x, base_channels=32): 
    a = Conv2D(base_channels*2, 1, 1, activation='relu')(x)

    b_1 = Conv2D(base_channels*4, 1, 1, activation='relu')(x) # 차원축소 
    b_2 = Conv2D(base_channels*4, 3, 1, padding='same', activation='relu')(b_1)

    c_1 = Conv2D(base_channels, 1, 1, activation='relu')(x) # 차원축소 
    c_2 = Conv2D(base_channels, 5, 1, padding='same', activation='relu')(c_1)

    d_1 = MaxPooling2D(3, 1, padding='same')(x)
    d_2 = Conv2D(base_channels, 1, 1, activation='relu')(d_1) # 차원축소 

    return Concatenate(axis=-1)([a, b_2, c_2, d_2]) 


- 위 코드 설명: 
    - 구글넷은 세 갈래의 **인셉션 모듈**을 사용함. 1x1 커널으로 차원 축
    - 1x1, 3x3, 5x5 세개의 Conv layer와 1개의 Max-pooling사용

**이 모듈을 여러 개 연결해서 구글넷을 만드는 것!**

In [5]:
# 구글넷 모델 
def googleNet_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # First Convolutional layer
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # First Inception module
    x = inception_module(x, base_channels=32)

    # Inception module 추가
    x = inception_module(x, base_channels=32)
    x = inception_module(x, base_channels=32)

    # Fully Connected layers
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output layer
    outputs = Dense(num_classes, activation='softmax')(x) 

    model = Model(inputs=inputs, outputs=outputs)
    return model


# GoogleNet 모델 학습 

In [6]:

# 이미지 전처리
data_dir = '/Users/eunjincho/Documents/workspaces/data_geeks/datasets/'
img_width, img_height = 224, 224

X, y = load_dataset(data_dir, img_width, img_height) # 224x224 사이즈로 변경 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 
model = googleNet_model((img_width, img_height, 3), len(os.listdir(data_dir)))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습 
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


Epoch 1/10
9/9 [==============================] - 274s 32s/step - loss: 3.8390 - accuracy: 0.1357 - val_loss: 1.9743 - val_accuracy: 0.1286
Epoch 2/10
9/9 [==============================] - 384s 43s/step - loss: 1.9979 - accuracy: 0.1964 - val_loss: 1.9900 - val_accuracy: 0.1143
Epoch 3/10
9/9 [==============================] - 233s 26s/step - loss: 1.9390 - accuracy: 0.2143 - val_loss: 1.9620 - val_accuracy: 0.1857
Epoch 4/10
9/9 [==============================] - 341s 40s/step - loss: 1.8711 - accuracy: 0.2286 - val_loss: 1.9338 - val_accuracy: 0.2000
Epoch 5/10
9/9 [==============================] - 314s 35s/step - loss: 1.7240 - accuracy: 0.3321 - val_loss: 1.8723 - val_accuracy: 0.2143
Epoch 6/10
9/9 [==============================] - 265s 30s/step - loss: 1.5663 - accuracy: 0.4679 - val_loss: 1.8966 - val_accuracy: 0.2286
Epoch 7/10
9/9 [==============================] - 360s 42s/step - loss: 1.3454 - accuracy: 0.5393 - val_loss: 1.8431 - val_accuracy: 0.2571
Epoch 8/10
9/9 [====

In [7]:

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)


3/3 [==============================] - 5s 1s/step - loss: 1.9201 - accuracy: 0.2571
Test loss: 1.9200844764709473
Test accuracy: 0.2571428716182709


# 하나의 이미지 분석해보기 

In [8]:
image_path = '/Users/eunjincho/Desktop/face1.jpeg'

input_image = preprocess_image(image_path, img_width, img_height)
input_image = np.expand_dims(input_image, axis=0) # 모델은 이미지 여러개를 입력 받도록 만들어졌는데,지금은 하나만 넣는 것이기 때문에 expand_dims 
predicted_probs = model.predict(input_image) # 감정들의 확률

predicted_emotion_index = np.argmax(predicted_probs) # 가장 맞을 확률이 높은 감정의 인덱스 
emotion_classes = sorted(os.listdir(data_dir))
predicted_emotion = emotion_classes[predicted_emotion_index] # 인덱스에 따른 감정을 변수로 저장 

print("Predicted emotion:", predicted_emotion)

1/1 [==============================] - 1s 551ms/step
Predicted emotion: sad


In [9]:
# 각 감정들의 확률 출력
for emotion, prob in zip(emotion_classes, predicted_probs[0]):
        percentage = prob * 100
        print(f"감정: {emotion},\t\t확률: {percentage:.2f}%")

감정: .DS_Store,		확률: 0.03%
감정: anger,		확률: 10.16%
감정: disgust,		확률: 10.39%
감정: fear,		확률: 7.65%
감정: happy,		확률: 1.75%
감정: neutral,		확률: 16.72%
감정: sad,		확률: 26.80%
감정: surprise,		확률: 26.49%
